In [11]:
import rasterio

#raster_path = r"E:\Sea Ice Classification\AI4ArcticASIP Sea ice Dataset version2\2018.910\subNC\tiff_polygon_icechart\20180903T082725_S1B_AMSR2_IcechartGreenlandNorthEast_Polygon.tif"
raster_path = r"E:\Sea Ice Classification\AI4ArcticASIP Sea ice Dataset version2\2018.910\subNC\tiff_SAR\20180903T082725_S1B_AMSR2_IcechartGreenlandNorthEast_SAR.tif"
with rasterio.open(raster_path) as dataset:
    print("CRS:", dataset.crs)  # Check CRS
    print("Transform:", dataset.transform)  # Check affine transform


CRS: None
Transform: | 1.00, 0.00, 0.00|
| 0.00, 1.00, 0.00|
| 0.00, 0.00, 1.00|


In [12]:
from osgeo import gdal

# Define file path
tif_path = raster_path
output_path =  r"E:\Sea Ice Classification\AI4ArcticASIP Sea ice Dataset version2\2018.910\subNC\tiff_SAR\20180903T082725_S1B_AMSR2_IcechartGreenlandNorthEast_SAR_CRS.tif" # Save as a new file

# Open dataset
dataset = gdal.Open(tif_path, gdal.GA_Update)

# Assign CRS (Make sure it's written)
gdal.Warp(output_path, dataset, dstSRS="EPSG:4326")

# Close dataset
dataset = None

print("CRS assigned and saved successfully!")


CRS assigned and saved successfully!


In [13]:
import rasterio

with rasterio.open(output_path) as dataset:
    print("CRS:", dataset.crs)  # Check CRS
    print("Transform:", dataset.transform)  # Check affine transform


CRS: EPSG:4326
Transform: | 0.00, 0.00,-30.10|
| 0.00,-0.00, 76.33|
| 0.00, 0.00, 1.00|


check shp coordinate

In [1]:
import rasterio
import fiona
from fiona.crs import from_epsg

#raster_path = r"E:\Sea Ice Classification\AI4ArcticASIP Sea ice Dataset version2\2018.910\subNC\tiff_polygon_icechart\20180903T082725_S1B_AMSR2_IcechartGreenlandNorthEast_Polygon.tif"
shp_path = r"C:\Users\TJ002\Desktop\Landfast ice data\shp file\Beaufort_sea_sample_region.shp"

# Check CRS of shapefile
with fiona.open(shp_path) as shp:
    print("CRS:", shp.crs)

# Define CRS if not present
with fiona.open(shp_path, 'r') as shp:
    if not shp.crs:
        shp.close()
        with fiona.open(shp_path, 'w', crs=from_epsg(4326)) as shp_write:
            print("CRS defined as EPSG:4326")
    else:
        print("CRS already defined")
    print("CRS:", shp.crs)  # Check CRS

CRS: EPSG:4326
CRS already defined
CRS: EPSG:4326


In [ ]:
import fiona
import json

# Define the output path for the GeoJSON file
geojson_path = shp_path.replace('.shp', '.json')

# Read the shapefile and write to GeoJSON
with fiona.open(shp_path, 'r') as shp:
    with fiona.open(geojson_path, 'w', driver='GeoJSON', crs=shp.crs, schema=shp.schema) as geojson:
        for feature in shp:
            geojson.write(feature)

print(f"Shapefile converted to GeoJSON and saved to {geojson_path}")

Shapefile converted to GeoJSON and saved to C:\Users\TJ002\Desktop\Landfast ice data\shp file\Greenland_North.geojson


In [3]:
import geopandas as gpd


gdf = gpd.read_file(shp_path)

# 保存为 JSON 文件（FeatureCollection 格式）
json_path = shp_path.replace('.shp', '.json')
gdf.to_file(json_path, driver="GeoJSON")

# Define the output path for the GeoJSON file
geojson_path = shp_path.replace('.shp', '.geojson')
from shapely.geometry import MultiPolygon

# 统一转换为 MultiPolygon
gdf["geometry"] = gdf["geometry"].apply(lambda geom: MultiPolygon([geom]) if geom.geom_type == "Polygon" else geom)
gdf.to_file(geojson_path, driver="GeoJSON")

print(f"转换完成，GeoJSON 文件保存在 {geojson_path}")


转换完成，GeoJSON 文件保存在 C:\Users\TJ002\Desktop\Landfast ice data\shp file\Beaufort_sea_sample_region.geojson


CS2eo无法识别geojson, still 识别不了

In [5]:
import json
gdf = gpd.read_file(shp_path)
# 构造 FeatureCollection
geojson_dict = {
    "type": "FeatureCollection",
    "features": json.loads(gdf.to_json())["features"]
}

# 保存为 GeoJSON 文件
with open(geojson_path, "w", encoding="utf-8") as f:
    json.dump(geojson_dict, f, ensure_ascii=False, indent=2)

print("转换完成，GeoJSON 文件已保存。")

转换完成，GeoJSON 文件已保存。
